# Data Exploration
Now that we have our data in MySQL, let us answer some basic questions with Python. We will again use SQLAlchemy to do this.

Running this whole notebook will prompt for your MySQL password and will only take a few seconds.

### Data Model
As a refresher, here is the database ERD that may help aid in coming up with some basic questions:
![msd-erd](msd-erd.png)

In [171]:
import math
import numpy as np
import os
import pandas as pd
import sqlalchemy as db

In [172]:
# DB connection set up
server = 'localhost'
database = 'songDB'
port = 3306
username = 'root'
password = input()
server_conn_string = f'mysql+mysqldb://{username}:{password}@{server}:{port}'
db_name = 'msd' # million song dataset
db_conn_string = server_conn_string + '/' + db_name
engine = db.create_engine(db_conn_string)
connection = engine.connect()

##### How big is our database in terms of storage? How does this compare to the size of the raw data subset?

In [173]:
statement = 'SELECT table_schema "msd", ROUND(SUM(data_length + index_length) / 1024 / 1024 / 1024, 1) "DB Size in GB" FROM information_schema.tables GROUP BY table_schema; '
ResultProxy = connection.execute(statement)
size_of_all_dbs = pd.DataFrame(ResultProxy.fetchall(), columns=('db', 'gb'))
print(size_of_all_dbs)

                   db   gb
0  information_schema  0.0
1                 msd  2.3
2               mysql  0.0
3  performance_schema  0.0
4                 sys  0.0


In [174]:
size_of_msd = float(size_of_all_dbs.gb[size_of_all_dbs.db.eq('msd')])
print(size_of_msd)

2.3


In [175]:
from pathlib import Path

root_directory = Path('MillionSongSubset')
raw_data_in_bytes = sum(f.stat().st_size for f in root_directory.glob('**/*') if f.is_file())
raw_data_in_gb = round(raw_data_in_bytes / 1000**3, 2)
print(raw_data_in_gb)
percent_used = round(size_of_msd / raw_data_in_gb * 100)
print(str(percent_used) + '%')

2.74
84%


The size of our raw data subset was 2.74 GB. During the database creation step, we extracted most of the information into MySQL, generating a database size of 2.3 GB. This represents 84% of the raw data subset.

##### How big are our tables in terms of rows, columns, and size?

In [176]:
meta = db.MetaData()
meta.reflect(bind=engine)
list_of_tables = list(meta.tables.keys())
print(list_of_tables)

['msd_artist', 'msd_artist_mbtag', 'msd_artist_similarity', 'msd_artist_term', 'msd_bar', 'msd_beat', 'msd_r_mbtag', 'msd_r_term', 'msd_section', 'msd_segment', 'msd_tatum', 'msd_track']


In [177]:
table_sizes = pd.DataFrame(columns=('table', 'row count', 'col count', 'size in MB'))

for table in list_of_tables:
    statement = f'SELECT COUNT(*) FROM {table};'
    ResultProxy = connection.execute(statement)
    row_count = ResultProxy.fetchall()[0][0]

    statement = f'SELECT COUNT(*) FROM information_schema.columns WHERE TABLE_SCHEMA = "msd" AND TABLE_NAME = "{table}";'
    ResultProxy = connection.execute(statement)
    col_count = ResultProxy.fetchall()[0][0]

    statement = f'SELECT ROUND((data_length + index_length) / 1024 / 1024, 3) FROM information_schema.tables WHERE table_schema = "msd" AND table_name = "{table}";'
    ResultProxy = connection.execute(statement)
    size_mb = float(ResultProxy.fetchall()[0][0])

    new_row = {'table': table, 'row count': row_count, 'col count': col_count, 'size in MB': size_mb}
    table_sizes = table_sizes.append(new_row, ignore_index=True)

In [178]:
print(table_sizes.sort_values(by='size in MB', ascending=False))

                    table row count col count  size in MB
9             msd_segment   8577406        32    1414.000
10              msd_tatum  10479481         5     538.000
5                msd_beat   4809945         5     267.766
4                 msd_bar   1649792         5      90.609
8             msd_section     99897         5       6.516
3         msd_artist_term     97493         3       5.516
2   msd_artist_similarity     42969         3       3.516
11              msd_track     10000        22       3.516
0              msd_artist      3888         5       0.391
1        msd_artist_mbtag      3804         3       0.234
7              msd_r_term      3502         2       0.156
6             msd_r_mbtag       721         2       0.063


The largest tables are msd_segment, msd_tatum, msd_beat, and msd_bar. This makes sense because for each song, there can be hundreds of bars, beats, and notes (segment/tatum).

##### How many artists span the 10,000 songs? If some artists have more than 1 song in this data subset, what is the distribution like?

In [179]:
statement = 'SELECT COUNT(*) FROM msd_artist;'
ResultProxy = connection.execute(statement)
artist_count = ResultProxy.fetchall()[0][0]
print(artist_count)

3888


There are 3,888 unique artists, meaning some artists have multiple songs in this data set. Below are the top 10 artists by song count in this data subset.

In [180]:
statement = 'WITH top_10_artist AS ('\
'SELECT artist_id, COUNT(*) AS song_count FROM msd_track GROUP BY artist_id ORDER BY song_count DESC LIMIT 10)'\
'SELECT a.artist_name, t.song_count FROM top_10_artist AS t LEFT JOIN msd_artist AS a ON t.artist_id = a.artist_id;'
ResultProxy = connection.execute(statement)
ResultProxy.fetchall()

[('Mario Rosenstock', 13),
 ('Snow Patrol', 12),
 ('The Jackson Southernaires', 12),
 ('Sugar Minott', 12),
 ('RUN-DMC', 12),
 ('Line Renaud', 12),
 ('Aerosmith', 12),
 ('Phil Collins', 12),
 ('Nick Cave & The Bad Seeds', 11),
 ('Stevie Ray Vaughan And Double Trouble', 11)]

##### What is the distribution of the songs' years?

In [181]:
statement = 'SELECT year, COUNT(*) AS year_count FROM msd_track GROUP BY year ORDER BY year_count DESC;'
ResultProxy = connection.execute(statement)
song_dist = pd.DataFrame(ResultProxy.fetchall(), columns=('year', 'song_count'))
print(song_dist)

      year  song_count
0      NaN        5320
1   2006.0         320
2   2005.0         304
3   2007.0         285
4   2004.0         270
..     ...         ...
64  1936.0           1
65  1934.0           1
66  1950.0           1
67  1929.0           1
68  1957.0           1

[69 rows x 2 columns]


In [182]:
missing_year = song_dist.song_count[song_dist.year.isna()][0]
print(missing_year)

5320


In [183]:
song_dist_clean = song_dist[song_dist.year.notna()]
print(min(song_dist_clean.year))
print(max(song_dist_clean.year))

1926.0
2010.0


In [184]:
print(song_dist_clean.iloc[:10])

      year  song_count
1   2006.0         320
2   2005.0         304
3   2007.0         285
4   2004.0         270
5   2003.0         254
6   2008.0         253
7   2009.0         250
8   2001.0         217
9   2002.0         198
10  2000.0         192


About half of the songs in our data subset have missing years. For songs that we do have years, the songs range from 1926 to 2010, with the most coming from the 00s.

In [185]:
song_dist.year.isna()

0      True
1     False
2     False
3     False
4     False
      ...  
64    False
65    False
66    False
67    False
68    False
Name: year, Length: 69, dtype: bool

##### What are the "hottest" songs in this subset?

In [186]:
statement = 'SELECT a.artist_name, t.title AS song, t.song_hotness '\
'FROM msd_track AS t INNER JOIN msd_artist AS a ON t.artist_id = a.artist_id WHERE t.song_hotness IS NOT NULL ORDER BY t.song_hotness DESC LIMIT 10;'
ResultProxy = connection.execute(statement)
song_hotness = pd.DataFrame(ResultProxy.fetchall(), columns=('artist', 'song', 'hotness'))
print(song_hotness)

                    artist                                               song  \
0                    B.o.B  Nothin' On You [feat. Bruno Mars] (Album Version)   
1             Led Zeppelin                     Immigrant Song (Album Version)   
2           Donny Hathaway                        This Christmas (LP Version)   
3               Nickelback         If Today Was Your Last Day (Album Version)   
4                 Maroon 5                                  Harder To Breathe   
5        The White Stripes                                        Blue Orchid   
6              Snow Patrol                                       Just Say Yes   
7  Pete Rock & C.L. Smooth           They Reminisce Over You (Single Version)   
8                     Muse             Exogenesis: Symphony Part 1 [Overture]   
9           The Mars Volta                                     Inertiatic Esp   

    hotness  
0  1.000000  
1  1.000000  
2  0.997758  
3  0.984347  
4  0.979837  
5  0.972387  
6  0.94599

In [187]:
connection.close

<bound method Connection.close of <sqlalchemy.engine.base.Connection object at 0x7f9b8688d640>>

### Future Considerations
- some columns seem to always be null, should we remove
- we skipped importing term/tag likelihoods, this could help with effectively designating a single genre per track
- still some normalization can be made
- some fields still have codes that don't have reference tables and need to be looked into (e.g. key, mode)